In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
from sklearn.preprocessing import MinMaxScaler
import keras
import plotly.io as pio
import plotly.graph_objects as go
import tensorflow as tf

In [ ]:
def create_dataset(signal_data, look_back=1):
    dataX, dataY = [], []
    for i in range(len(signal_data)-look_back):
        dataX.append(signal_data[i:(i+look_back), 0])
        dataY.append(signal_data[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

def ratio(val, rate):
    result = val + val*rate
    if result < 0:
        result = 0
    if result > 1:
        result = 1
    return result

CSV_FILE_PATH = "D:\\workspace\\python\\sample\\train-0001-0420.csv"
data = pd.read_csv(CSV_FILE_PATH)

In [ ]:
# data.ScaleLoad[300:800] += 0.2
data['OrgLoad'] = data['Load']
data['Load'][300:800] = data['Load'][300:800].apply(lambda x : ratio(x, 0.4))
data.Load[1274:1447] = 0.4

In [ ]:
print(data.info())

In [ ]:
pio.templates.default = "plotly_dark"


fig = go.Figure(layout=go.Layout(height=600, width=1200))
fig = fig.add_trace(go.Scatter(x=data.index[:],
                               y=data.Load[:],
                               mode='lines',
                               name="Changed Spindle Load"))
fig = fig.add_trace(go.Scatter(x=data.index[:],
                               y=data.OrgLoad[:],
                               mode='lines',
                               name="Origin Spindle Load"))
fig.update_layout(title='Spindle Load',
                   xaxis_title='time',
                   yaxis_title='load')
fig.show()

In [ ]:
x_test, y_test = create_dataset(data['Load'].values[:, None], 100)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
model = tf.keras.models.load_model('D:\\workspace\\python\\sample\\lstm-model-069-021-0.000129.h5')

In [ ]:
x_test.shape

In [ ]:
# 100개의 데이터로 1개 데이터 예측을 반복
look_ahead = 1900

predictions = model.predict(x_test[:look_ahead], batch_size=1, verbose=1)

revel_predictions = np.ravel(predictions, order='C')
revel_y_test = np.ravel(y_test[:look_ahead], order='C')

fig = go.Figure(layout=go.Layout(height=600, width=1200))
fig = fig.add_trace(go.Scatter(y=y_test[:look_ahead],
                               mode='lines',
                               name="origin"))
fig = fig.add_trace(go.Scatter(y=revel_predictions,
                               mode='lines',
                               name="predict"))
fig.show()


In [ ]:
print(x_test.shape)
PREDICT_NUM = 2

predictions = []
for i in range(0, x_test.shape[0], PREDICT_NUM):
    x_input = x_test[i]
    for j in range(PREDICT_NUM):
        prediction = model.predict(np.array([x_input]), batch_size=1, verbose=0)
        predictions.append(prediction)
        x_input = np.vstack([x_input[1:], prediction])

revel_predictions = np.ravel(predictions, order='C')
revel_y_test = np.ravel(y_test, order='C')


In [ ]:
fig = go.Figure(layout=go.Layout(height=600, width=1200))
fig = fig.add_trace(go.Scatter(y=y_test,
                               mode='lines',
                               name="origin"))
fig = fig.add_trace(go.Scatter(y=revel_predictions,
                               mode='lines',
                               name="predict"))
fig.show()

In [ ]:
# 100개의 실 데이터로 5개 데이터 예측을 반복

# look_ahead = 5

# x_input = x_test[0]
# predictions = []

# print('x_input \n', x_input)
# prediction = model.predict(np.array([x_input]), batch_size=1, verbose=2)
# print('prediction \n', prediction)
# predictions.append(prediction)

# x_input = np.vstack([x_input[1:], prediction])
# print('x_input \n', x_input)
# prediction = model.predict(np.array([x_input]), batch_size=1, verbose=2)
# predictions.append(prediction)


# print('predictions \n', predictions)

# # print(np.ravel(prediction, order='C'))
# ###################################################

# # 최초 예측할 100개의 데이터
# x_input = x_test[0]


In [ ]:
# plt.figure(figsize=(12,5))
# plt.plot(np.arange(look_ahead),predictions,'r',label="prediction")
# plt.plot(np.arange(look_ahead),y_test[:look_ahead],label="test function")
# plt.legend()
# plt.show()

In [ ]:
# look_ahead = 10
# xhat = x_test[0]
# predictions = np.zeros((look_ahead,1))
# for i in range(look_ahead):
#     prediction = model.predict(np.array([xhat]), batch_size=1)
#     predictions[i] = prediction
#     xhat = np.vstack([xhat[1:],prediction])